In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [2]:
n_hidden = 35
lr = 0.01
epochs = 1000

string = "hello pytorch. how long can a rnn cell remember?"
chars = "abcdefghifklmnopqrstuvwxyz ?!.,:;01"
char_list = [i for i in chars]
n_letters = len(char_list) #unique 

In [3]:
def string_to_onehot(string):
    start = np.zeros(shape=len(char_list),dtype=int) # 1D
    end = np.zeros(shape=len(char_list),dtype=int)
    start[-2] = 1    # To indicate
    end[-1] = 1
    for i in string:
        idx = char_list.index(i)
        zero = np.zeros(shape=n_letters,dtype=int)
        zero[idx]=1
        start = np.vstack([start,zero]) # 2D
    output = np.vstack([start,end])
    return output


In [8]:
def onehot_to_word(onehot_1):
    onehot = torch.Tensor.numpy(onehot_1)
    return char_list[onehot.argmax()]

In [5]:
torch.Tensor.numpy(torch.Tensor([1,2,3])) #torch.Tensor([1,2,3]) 's type == torch.Tensor

array([1., 2., 3.], dtype=float32)

In [7]:
type(torch.Tensor([1,2,3]))  

torch.Tensor

In [10]:
class RNN(nn.Module):
    def __init__(self,input_size,hidden_size,output_size):
        super(RNN,self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        self.i2h = nn.Linear(input_size,hidden_size)
        self.h2h = nn.Linear(hidden_size,hidden_size)
        self.i2o = nn.Linear(hidden_size,output_size)
        self.act_fn = nn.Tanh()
        
    def forward(self,input,hidden):
        hidden = self.act_fn(self.i2h(input)+self.h2h(hidden))
        output = self.i2o(hidden)
        return output, hidden
    
    def init_hidden(self):
        return torch.zeros(1,self.hidden_size)
    
rnn = RNN(n_letters,n_hidden,n_letters)

In [11]:
loss_func = nn.MSELoss()
optimizer = torch.optim.Adam(rnn.parameters(),lr=lr)

In [12]:
one_hot = torch.from_numpy(string_to_onehot(string)).type_as(torch.FloatTensor())

In [16]:
for i in range(epochs):
    rnn.zero_grad() # == optimizer.zero_grad()
    total_loss = 0
    hidden = rnn.init_hidden()
    
    for j in range(one_hot.size()[0]-1):
        input_ = one_hot[j:j+1,:]
        target = one_hot[j+1]
        
        output, hidden = rnn.forward(input_,hidden)
        loss = loss_func(output.view(-1),target.view(-1)) # squeeze dim
        total_loss += loss
        input_ = output
    
    total_loss.backward()
    optimizer.step()
    
    if i % 10 == 0:
        print(total_loss)

tensor(2.0840, grad_fn=<AddBackward0>)
tensor(0.9111, grad_fn=<AddBackward0>)
tensor(0.5772, grad_fn=<AddBackward0>)
tensor(0.3717, grad_fn=<AddBackward0>)
tensor(0.2442, grad_fn=<AddBackward0>)
tensor(0.1840, grad_fn=<AddBackward0>)
tensor(0.1616, grad_fn=<AddBackward0>)
tensor(0.1377, grad_fn=<AddBackward0>)
tensor(0.1155, grad_fn=<AddBackward0>)
tensor(0.1024, grad_fn=<AddBackward0>)
tensor(0.0875, grad_fn=<AddBackward0>)
tensor(0.0772, grad_fn=<AddBackward0>)
tensor(0.0680, grad_fn=<AddBackward0>)
tensor(0.0738, grad_fn=<AddBackward0>)
tensor(0.0586, grad_fn=<AddBackward0>)
tensor(0.0530, grad_fn=<AddBackward0>)
tensor(0.0504, grad_fn=<AddBackward0>)
tensor(0.0430, grad_fn=<AddBackward0>)
tensor(0.0426, grad_fn=<AddBackward0>)
tensor(0.0370, grad_fn=<AddBackward0>)
tensor(0.0330, grad_fn=<AddBackward0>)
tensor(0.0305, grad_fn=<AddBackward0>)
tensor(0.0339, grad_fn=<AddBackward0>)
tensor(0.0276, grad_fn=<AddBackward0>)
tensor(0.0262, grad_fn=<AddBackward0>)
tensor(0.0232, grad_fn=<A

In [19]:
start = torch.zeros(1,len(char_list))
start[:,-2] = 1

with torch.no_grad():
    hidden = rnn.init_hidden()
    input_ = start
    output_string = ""
    for i in range(len(string)):
        output,hidden = rnn.forward(input_,hidden)
        output_string += onehot_to_word(output.data)
        input_ = output
        
print(output_string)

hello mecbber  abberpy ebbbern emboerh  mbeblpa 
